In [95]:
import numpy as np
import pandas as pd
import re

from sklearn.metrics import roc_auc_score, confusion_matrix

In [96]:
# Load your dataset
X_train_df = pd.read_csv('data_train_infection.csv')
# X_train_df = pd.read_csv('data_train_rebleeding.csv')
# X_train_df = pd.read_csv('data_train_mortality.csv')

# Define target column
target_col = "infection"
# target_col = "rebleeding"
# target_col = "mortality"

load the score table

In [97]:
score_card_df = pd.read_csv('autoscore_output.csv')
score_card_df = score_card_df[1::2]
print(score_card_df.shape)
score_card_df

(20, 2)


,Variable_Category,Score
1,Ascitesnot_0,2
3,Bilirubin_mg_dL__bin_00,2
5,Na_mEq_L__bin_0not_0,6
7,Etiology_of_cirrhosis_HCVnot_0,4
9,HCCnot_0,1
11,Sex1,2
13,Albumin_g_dL__bin_1not_1,7
15,Systolic_blood_pressure_mmHg__bin_01,2
17,Age_bin_1not_0,1
19,Na_mEq_L__bin_1not_1,9


In [98]:
score_card_df[['variable_name', 'suffix']] = score_card_df['Variable_Category'].str.extract(r'^(.*?)(not_0|not_1|0|1)$')
score_card_df

,Variable_Category,Score,variable_name,suffix
1,Ascitesnot_0,2,Ascites,not_0
3,Bilirubin_mg_dL__bin_00,2,Bilirubin_mg_dL__bin_0,0
5,Na_mEq_L__bin_0not_0,6,Na_mEq_L__bin_0,not_0
7,Etiology_of_cirrhosis_HCVnot_0,4,Etiology_of_cirrhosis_HCV,not_0
9,HCCnot_0,1,HCC,not_0
11,Sex1,2,Sex,1
13,Albumin_g_dL__bin_1not_1,7,Albumin_g_dL__bin_1,not_1
15,Systolic_blood_pressure_mmHg__bin_01,2,Systolic_blood_pressure_mmHg__bin_0,1
17,Age_bin_1not_0,1,Age_bin_1,not_0
19,Na_mEq_L__bin_1not_1,9,Na_mEq_L__bin_1,not_1


In [99]:
# Initialize an array to hold the total score for each patient.
total_scores = np.zeros(len(X_train_df))

# Loop over each rule in the scorecard
for idx, row in score_card_df.iterrows():
    var = row['variable_name']  # base variable name from the scorecard
    rule_suffix = row['suffix']  # one of "not_0", "not_1", "0", "1"
    rule_score = row['Score']    # points assigned for this rule
    
    # If the variable exists in the patient data, apply the rule.
    if var in X_train_df.columns:
        # Get the binary values for this variable for every patient.
        values = X_train_df[var].values  # e.g. an array of 0's and 1's
        
        # Apply the rule based on suffix
        if rule_suffix in ['not_0', '1']:
            # If the rule requires a value of 1, create a mask where the value is 1.
            mask = (values == 1)
        elif rule_suffix in ['not_1', '0']:
            # If the rule requires a value of 0, create a mask where the value is 0.
            mask = (values == 0)
        
        # Add the score for each patient where the condition holds.
        total_scores += mask.astype(float) * rule_score
    else:
        # If the variable from the scorecard is not found in X_train_df,
        # then it doesn't contribute to the score.
        continue

print("Total risk scores for the first 10 patients:")
print(total_scores[:10])


Total risk scores for the first 10 patients:
[30. 38. 32. 47. 24. 61. 35. 54. 30. 27.]


In [100]:
scores_train = total_scores
labels_train = X_train_df[target_col]

min_thresh = 0
max_thresh = int(np.ceil(score_card_df["Score"].sum()))
print(f"Threshold range: {min_thresh} to {max_thresh}")
thresholds = range(min_thresh, max_thresh + 1)

results = []

for thresh in thresholds:
    # Predict: assign 1 if the score is >= threshold, otherwise -1.
    preds = np.where(scores_train >= thresh, 1, -1)

    # Compute confusion matrix using labels in the order: negative (-1) then positive (1)
    tn, fp, fn, tp = confusion_matrix(labels_train, preds, labels=[-1, 1]).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    youden_index = sensitivity + specificity - 1

    results.append({
        "threshold": thresh,
        "youden_index": youden_index,
        "sensitivity": sensitivity,
        "specificity": specificity,
        "precision": tp / (tp + fp) if (tp + fp) > 0 else 0,
        "npv": tn / (tn + fn) if (tn + fn) > 0 else 0,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    })

# Convert results to a DataFrame and find the best threshold using the Youden index.
results_df = pd.DataFrame(results)
best_result = results_df.loc[results_df["youden_index"].idxmax()]

print("Best threshold based on Youden Index on the training set:")
print(best_result)

Threshold range: 0 to 103
Best threshold based on Youden Index on the training set:
threshold        55.000000
youden_index      0.297514
sensitivity       0.413793
specificity       0.883721
precision         0.155844
npv               0.966732
tp               12.000000
tn              494.000000
fp               65.000000
fn               17.000000
Name: 55, dtype: float64


In [101]:
thresh = best_result['threshold']

preds_train = np.where(scores_train >= thresh, 1, -1)

tn, fp, fn, tp = confusion_matrix(labels_train, preds_train, labels=[-1, 1]).ravel()

accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
npv = tn / (tn + fn) if (tn + fn) > 0 else 0

# F1 scores for both classes
f1_pos = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
f1_neg = 2 * (specificity * npv) / (specificity + npv) if (specificity + npv) > 0 else 0
f1_macro = (f1_pos + f1_neg) / 2

# AUC Score
# labels_test should be {1, -1}, convert to {1, 0} for AUC
labels_binary = (labels_train == 1).astype(int)
auc = roc_auc_score(labels_binary, scores_train)

print("=== Train Set Evaluation ===")
print(f"Threshold: {thresh}")
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Sensitivity (Recall): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Negative Predictive Value: {npv:.3f}")
print(f"F1 Macro Score: {f1_macro:.3f}")
print(f"AUC Score: {auc:.3f}")

=== Train Set Evaluation ===
Threshold: 55.0
TP: 12, FP: 65, TN: 494, FN: 17
Accuracy: 0.861
Sensitivity (Recall): 0.414
Specificity: 0.884
Precision: 0.156
Negative Predictive Value: 0.967
F1 Macro Score: 0.575
AUC Score: 0.703


In [102]:
# Load test dataset
X_test_df = pd.read_csv('data_test_infection.csv')
# df_test = pd.read_csv('data_test_rebleeding.csv')
# df_test = pd.read_csv('data_test_mortality.csv')

In [103]:
# Initialize an array to hold the total score for each patient.
total_scores = np.zeros(len(X_test_df))

# Loop over each rule in the scorecard
for idx, row in score_card_df.iterrows():
    var = row['variable_name']  # base variable name from the scorecard
    rule_suffix = row['suffix']  # one of "not_0", "not_1", "0", "1"
    rule_score = row['Score']    # points assigned for this rule
    
    # If the variable exists in the patient data, apply the rule.
    if var in X_test_df.columns:
        # Get the binary values for this variable for every patient.
        values = X_test_df[var].values  # e.g. an array of 0's and 1's
        
        # Apply the rule based on suffix
        if rule_suffix in ['not_0', '1']:
            # If the rule requires a value of 1, create a mask where the value is 1.
            mask = (values == 1)
        elif rule_suffix in ['not_1', '0']:
            # If the rule requires a value of 0, create a mask where the value is 0.
            mask = (values == 0)
        
        # Add the score for each patient where the condition holds.
        total_scores += mask.astype(float) * rule_score
    else:
        # If the variable from the scorecard is not found in X_test_df,
        # then it doesn't contribute to the score.
        continue

print("Total risk scores for the first 10 patients:")
print(total_scores[:10])

Total risk scores for the first 10 patients:
[33. 61. 52. 45. 69. 26. 41. 50. 68. 59.]


In [104]:
scores_test = total_scores
labels_test = X_test_df[target_col]
thresh = best_result['threshold']

preds_test = np.where(scores_test >= thresh, 1, -1)

tn, fp, fn, tp = confusion_matrix(labels_test, preds_test, labels=[-1, 1]).ravel()

accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
npv = tn / (tn + fn) if (tn + fn) > 0 else 0

# F1 scores for both classes
f1_pos = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
f1_neg = 2 * (specificity * npv) / (specificity + npv) if (specificity + npv) > 0 else 0
f1_macro = (f1_pos + f1_neg) / 2

# AUC Score
# labels_test should be {1, -1}, convert to {1, 0} for AUC
labels_binary = (labels_test == 1).astype(int)
auc = roc_auc_score(labels_binary, scores_test)

print("=== Test Set Evaluation ===")
print(f"Threshold: {thresh}")
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Sensitivity (Recall): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Negative Predictive Value: {npv:.3f}")
print(f"F1 Macro Score: {f1_macro:.3f}")
print(f"AUC Score: {auc:.3f}")

=== Test Set Evaluation ===
Threshold: 55.0
TP: 3, FP: 32, TN: 208, FN: 10
Accuracy: 0.834
Sensitivity (Recall): 0.231
Specificity: 0.867
Precision: 0.086
Negative Predictive Value: 0.954
F1 Macro Score: 0.517
AUC Score: 0.592
